In [ ]:
import findspark
findspark.init()

#1a01e0e77d803e8e292dbb639aa77646d41947892fe6f227

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://a92682f5ddcc:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/11 11:41:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from grandeInformacion.spark_utils import SparkUtils

logs = spark \
                .readStream \
                .format("text") \
                .option("cleanSource", "archive") \
                .option("sourceArchiveDir", "/home/jovyan/archive") \
                .load("/home/jovyan/notebooks/data/logs_output/")

In [4]:
from pyspark.sql.functions import split, trim


logs = logs.withColumn("value", trim(logs["value"]))

logs_df = logs.select(split(logs.value, "\\|").alias("log_array"))


logs_df = logs_df.withColumn("date", logs_df["log_array"].getItem(0).cast("string"))
logs_df = logs_df.withColumn("type", logs_df["log_array"].getItem(1).cast("string"))
logs_df = logs_df.withColumn("message", logs_df["log_array"].getItem(2).cast("string"))
logs_df = logs_df.withColumn("node", logs_df["log_array"].getItem(3).cast("string"))

logs_df.printSchema()


root
 |-- log_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- date: string (nullable = true)
 |-- type: string (nullable = true)
 |-- message: string (nullable = true)
 |-- node: string (nullable = true)



In [5]:
logs_df = logs_df.filter(logs_df["type"] == "ERROR")

In [6]:
query = logs_df \
    .writeStream \
    .outputMode("append") \
    .trigger(processingTime='25 seconds') \
    .option("truncate", "false") \
    .format("console") \
    .start()

query.awaitTermination(200)

25/04/11 11:41:47 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-44659530-1e13-4a9d-b96d-7e2bf4922b12. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/11 11:41:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|log_array                                                             |date               |type |message                  |node         |
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|[06/04/2025 06:36:57, ERROR, 500 Internal Server Error, server-node-2]|06/04/2025 06:36:57|ERROR|500 Internal Server Error|server-node-2|
|[06/04/2025 06:36:59, ERROR, 500 Internal Server Error, server-node-2]|06/04/2025 06:36:59|ERROR|500 Internal Server Error|server-node-2|
|[06/04/2025 06:37:01, ERROR, 500 Internal Server Error, server-node-2]|06/04/2025 06:37:01|ERROR|500 Internal Server Error|server-node-2|
|[06/04/2025 06:37:03, ERROR, 500 Internal Server Error, server-node-

False

In [7]:
query.stop()

In [8]:
sc.stop()